In [ ]:
import random, subprocess, json, time, os
import numpy as np
from tqdm import tqdm

In [ ]:
# number of matches to run
num_matches = 3

# Print win records between matches
PRINT_PROGRESS = False

In [ ]:
# read in ai bot models to count them
ais = os.listdir('./models')
ais = [ai for ai in ais if ai.endswith('.h5')]

# how many ai models
num_versions = len(ais)
ai_versions = list(range(num_versions))

In [ ]:
# list of non-ai enemy bots to choose from
bot_list = []

In [ ]:
# add manually adjusted bots (quantity <> probability of use)
bot_list.extend([
    "mining","mining","mining",
    "mine_attack", "mine_attack","mine_attack","mine_attack",
    "aggressive", "aggressive","aggressive","aggressive",
    "mine_attack_later", "mine_attack_later","mine_attack_later","mine_attack_later",
    ])

In [ ]:
# add random bots (quantity <> probability of use)
bot_list.extend([
    "random0","random0",
    "random1","random1",
    "random3","random3",
    "random5","random5",
    "random10","random10",
    "random15","random15",
    "random20","random20",
    ])

In [ ]:
# load win records from file
def load_wins(file='wins.txt'):
    if os.path.isfile(file):
        with open(file, 'r') as f:
            lines = f.readlines()
            wins = eval(lines[0])
            num = eval(lines[1])
    else:
        # track wins per ai model
        wins = dict({i:0 for i in list(range(num_versions))+list(set(bot_list))})
        # track matches per ai model
        num = dict({i:0 for i in list(range(num_versions))+list(set(bot_list))})
    return wins, num

# save win records to file
def save_wins(file='wins.txt'):
    with open(file, 'w') as f:
        f.write(str(wins)+'\n')
        f.write(str(num)+'\n')

# load initially
wins, num = load_wins()

In [ ]:
# clean up .log, .vec, and replay files
def cleanup():
    files = os.listdir('.')
    for file in files:
        if file.endswith('.vec') or file.endswith('.log') or file.startswith('replay'):
            os.remove(file)

In [ ]:
# add training data to train.in, train.out
def append_training(inputs, outputs):
    # check lengths, append to training data
    if len(input_lines)==len(output_lines):
        with open("train.in", "a") as f:
            for l in input_lines:
                f.write(l)
        with open("train.out", "a") as f:
            for l in output_lines:
                f.write(l)
        return True
    return False

In [ ]:
# read data from winner's .vec files
def read_training(winner_id, winner):
    # get inputs
    with open("{}_{}_input.vec".format(winner_id, winner), "r") as f:
        input_lines = f.readlines()

    # get outputs
    with open("{}_{}_out.vec".format(winner_id, winner), "r") as f:
        output_lines = f.readlines()
    
    return input_lines, output_lines

In [ ]:
# get string of bot win records for printing
def progress():
    out = ''
    for idx, i in enumerate(ai_versions+list(set(bot_list))):
        if np.mod(idx,4)==0:
            out += '\n'
        try:
            out += '{}:{}/{}={}%\t'.format(i,wins[i],num[i],round(100*wins[i]/num[i],1))
        except ZeroDivisionError:
            out += '{}:{}/0=0%\t'.format(i,wins[i])
    return out

In [ ]:
# make command string to call the executable with bots
def generate_cmd():
    g2g = False  # g2g flag used to make sure there's at least 1 ai bot in each battle
    
    while not g2g:
        # call the game executable
        cmd = 'halite.exe'

        # store enemy type (file name or ai version)
        enemy_version = []

        # list of ai versions to choose from
        ai_versions = list(range(num_versions))
        
        # add 2 or 4 players
        for _ in range(random.choice([2, 4])):
            # randomly choose from from ai or botlist
            enemy_version.append(random.choice(ai_versions+['na']*len(bot_list)))

            # build player into cmd string
            if enemy_version[-1] == 'na':
                # bot_list player
                player = random.choice(bot_list)  # choose from list
                cmd+=' "python ../bots/{}.py"'.format(player)  # append player to cmd string
                enemy_version[-1] = player  # record the bot
            else:
                # ai player
                cmd+=' "python ../bots/MyBot.py "{}""'.format(enemy_version[-1])
                g2g = True
    
    # finish cmd string with custom game settings, no time-outs, and quite mode(required)
    cmd+=' -q -t --constantsfile "constants.txt"'
    
    return cmd, enemy_version

In [ ]:
cleanup()
wins, num = load_wins()

# run number of matches or until stop.txt exists (whichever happens first)
for m in tqdm(range(num_matches)):
    # make command string to call the executable with bots
    cmd, enemy_versions = generate_cmd()
    
    # run match
    result = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout.read()
    # translate results from json
    try:
        result = json.loads(result)
    except:
        # cannot load as json
        print(result)
        break
    
    # ignore matches where there's an error
    if not len(result['error_logs']):
        # increment how many matches each bot has been in
        for ev in enemy_versions:
            num[ev] += 1
        
        # sort players by rank
        ranked = sorted(result['stats'].items(), key=lambda t:t[1]['rank'])
        
        # get best ranked player
        for player in ranked:
            winner_id = int(player[0])
            winner = enemy_versions[winner_id]
            break
        
        # read winner's data
        input_lines, output_lines = read_training(winner_id, winner)
        
        # check lengths, append to training data
        if not append_training(input_lines, output_lines):
            raise Exception('Training data lengths do not match')
        
        # increment wins for winner
        wins[winner] += 1
        
        # print as we go
        if PRINT_PROGRESS:
            tqdm.write('{} id {} won in {}'.format(winner, winner_id, enemy_versions))
            tqdm.write(result['replay'])
            tqdm.write("\nWins"+progress())
        
        save_wins()
        cleanup()
        
        # stop the loop externally
        if os.path.isfile('stop.txt'):
            break
    
    # if result['error_logs']:
    else:
        print("\nWins")
        print_progress()
        
        print(result)
        print(enemy_versions)
        break
    
    time.sleep(0.5)
    
print('\n****** FINAL SCORES ******')
print(progress())